# Document Retrival with Title Embedding and IDF on Texts (DR.TEIT)

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

## Dataset
### Dataset Description

- **mutldoc2dial_doc.json** contains the documents that are indexed by key `domain` and `doc_id` . Each document instance includes the following,

  - `doc_id`: the ID of a document;
  - `title`: the title of the document;
  - `domain`: the domain of the document;
  - `doc_text`: the text content of the document (without HTML markups);
  - `doc_html_ts`: the document content with HTML markups and the annotated spans that are indicated by `text_id` attribute, which corresponds to `id_sp`.
  - `doc_html_raw`: the document content with HTML markups and without span annotations.
  - `spans`: key-value pairs of all spans in the document, with `id_sp` as key. Each span includes the following,
    - `id_sp`: the id of a  span as noted by `text_id` in  `doc_html_ts`;
    - `start_sp`/  `end_sp`: the start/end position of the text span in `doc_text`;
    - `text_sp`: the text content of the span.
    - `id_sec`: the id of the (sub)section (e.g. `<p>`) or title (`<h2>`) that contains the span.
    - `start_sec` / `end_sec`: the start/end position of the (sub)section in `doc_text`.
    - `text_sec`: the text of the (sub)section.
    - `title`: the title of the (sub)section.
    - `parent_titles`: the parent titles of the `title`.

- **multidoc2dial_dial_train.json** and **multidoc2dial_dial_validation.json**  contain the training and dev split of dialogue data that are indexed by key `domain` . Please note: **For test split, we only include a dummy file in this version.**

  Each dialogue instance includes the following,

  - `dial_id`: the ID of a dialogue;
  - `turns`: a list of dialogue turns. Each turn includes,
    - `turn_id`: the time order of the turn;
    - `role`: either "agent" or "user";READ
    - `da`: dialogue act;
    - `references`: a list of spans with `id_sp` ,  `label` and `doc_id`. `references` is empty if a turn is for indicating previous user query not answerable or irrelevant to the document. **Note** that labels "*precondition*"/"*solution*" are fuzzy annotations that indicate whether a span is for describing a conditional context or a solution.
    - `utterance`: the human-generated utterance based on the dialogue scene.
Downloading the training dataset:

In [3]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!gdown --id 1Ln4pU93_ofAkbrz1uibsNABB0QsEaOXw

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ln4pU93_ofAkbrz1uibsNABB0QsEaOXw
To: /content/multidoc2dial.zip
100% 6.45M/6.45M [00:00<00:00, 49.9MB/s]


In [5]:
!unzip multidoc2dial.zip

Archive:  multidoc2dial.zip
replace multidoc2dial/multidoc2dial_dial_validation.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
def clean_text(text):
    """
    Clean the given text.

    :param text: input text
    :type text: str
    :return: cleaned string
    """
    return text.strip()

In [7]:
import json
with open('multidoc2dial/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

### Extracting titles

In [8]:
parent_titles = []
titles = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
            titles.append(multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans'][id_sp1]['title']) 
            parent_titles.append(doc_idx2)
          

In [9]:
len(titles)

35659

### Extracting document texts

In [10]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        title_to_domain[doc_idx2] = doc_idx1
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
             s = (multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['spans'][id_sp1]['text_sec'].strip())
             s = s.replace('!', '.')
             s = s.replace('! ', '.')
             s = s.replace('?', '.')
             s = s.replace('? ', '.')
             s = s.replace('\n', '.')
             m = s.split('.')
             #print(m)
             S = ""
             func = lambda w: w[:1].lower() + w[1:] if w else ''
             for word in m:
                 S = S + func(word) + ' '
             doc_texts_train.append(S)
            
doc_texts_train[10]

'the number of credits needed to provide benefits for your survivors depends on your age when you die  No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit  But , the younger a person is , the fewer credits they must have for family members to receive survivors benefits  '

In [11]:
len(doc_texts_train)

35659

## Encoding the sentences
We use the LaBSE which is a Language-agnostic BERT Sentence Encoder (LaBSE) is a BERT-based model trained for sentence embedding for 109 languages. The pre-training process combines masked language modeling with translation language modeling. The model is useful for getting multilingual sentence embeddings and for bi-text retrieval.

In [12]:
!pip install --quiet transformers

In [13]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import torch
from torch.nn.functional import normalize

In [14]:
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE")

### `get_embeddings`
In this method we extract the **pooler output** (Last layer hidden-state of the first token of the sequence (classification token) after further processing through the layers used for the auxiliary pretraining task. E.g. for BERT-family of models, this returns the classification token after processing through a linear layer and a tanh activation function. The linear layer weights are trained from the next sentence prediction (classification) objective during pretraining).

In [15]:
def get_embeddings(sentece):
    """
    Return embeddings based on encoder model

    :param sentence: input sentence(s)
    :type sentence: str or list of strs
    :return: embeddings
    """
    tokenized = tokenizer_labse(sentece,
                                return_tensors="pt",
                                padding=True)
    with torch.no_grad():
        embeddings = model_labse(**tokenized)
    
    return np.squeeze(np.array(embeddings.pooler_output))

### Title embedding

In [17]:
title_embeddings = []
progress = 0
TRAIN_SIZE = len(titles)
prev_title = ''
i = 0
for title in titles:
    if(title == prev_title):
        embd = title_embeddings[i-1]
    else:
        embd = get_embeddings(title) 
    title_embeddings.append(embd)
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))
    prev_title = title
    i += 1
 

Progress Percent = 0.14021705600269216%
Progress Percent = 0.2804341120053843%
Progress Percent = 0.4206511680080765%
Progress Percent = 0.5608682240107686%
Progress Percent = 0.7010852800134608%
Progress Percent = 0.841302336016153%
Progress Percent = 0.9815193920188452%
Progress Percent = 1.1217364480215373%
Progress Percent = 1.2619535040242296%
Progress Percent = 1.4021705600269216%
Progress Percent = 1.542387616029614%
Progress Percent = 1.682604672032306%
Progress Percent = 1.822821728034998%
Progress Percent = 1.9630387840376904%
Progress Percent = 2.1032558400403825%
Progress Percent = 2.2434728960430745%
Progress Percent = 2.383689952045767%
Progress Percent = 2.523907008048459%
Progress Percent = 2.664124064051151%
Progress Percent = 2.8043411200538433%
Progress Percent = 2.9445581760565354%
Progress Percent = 3.084775232059228%
Progress Percent = 3.22499228806192%
Progress Percent = 3.365209344064612%
Progress Percent = 3.505426400067304%
Progress Percent = 3.645643456069996

In [18]:
with open('doc_title_LaBSE_Embedding.npy', 'wb') as f:
    np.save(f, np.array(title_embeddings))

In [19]:
title_to_embeddings = {}
progress = 0
TRAIN_SIZE = len(titles)
prev_title = ''
for title in titles:
    if(title == prev_title):
       title_to_embeddings[title] = title_to_embeddings[prev_title]
    else:
       title_to_embeddings[title] = get_embeddings(title)
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))
    prev_title = title

Progress Percent = 0.14021705600269216%
Progress Percent = 0.2804341120053843%
Progress Percent = 0.4206511680080765%
Progress Percent = 0.5608682240107686%
Progress Percent = 0.7010852800134608%
Progress Percent = 0.841302336016153%
Progress Percent = 0.9815193920188452%
Progress Percent = 1.1217364480215373%
Progress Percent = 1.2619535040242296%
Progress Percent = 1.4021705600269216%
Progress Percent = 1.542387616029614%
Progress Percent = 1.682604672032306%
Progress Percent = 1.822821728034998%
Progress Percent = 1.9630387840376904%
Progress Percent = 2.1032558400403825%
Progress Percent = 2.2434728960430745%
Progress Percent = 2.383689952045767%
Progress Percent = 2.523907008048459%
Progress Percent = 2.664124064051151%
Progress Percent = 2.8043411200538433%
Progress Percent = 2.9445581760565354%
Progress Percent = 3.084775232059228%
Progress Percent = 3.22499228806192%
Progress Percent = 3.365209344064612%
Progress Percent = 3.505426400067304%
Progress Percent = 3.645643456069996

In [20]:
import pickle
with open('title_to_embeddings.pkl', 'wb') as f:
    pickle.dump(title_to_embeddings, f)

## Calculating the IDF for each token

In [21]:
words = set()
doc_texts_train_tokenized = []
for doc in doc_texts_train:
    tokenized_doc = [s.lower() for s in tokenizer_labse.tokenize(doc)]
    doc_texts_train_tokenized.append(tokenized_doc) 
    words = set(tokenized_doc).union(words)
len(words)

8477

In [22]:
words2IDF = {}
N_doc = len(doc_texts_train)
for i, word in enumerate(words):
    n_word = 0
    for doc in doc_texts_train_tokenized:
        if word in doc:
            n_word += 1
    words2IDF[word] = np.log(N_doc / (n_word + 1))
    if i % 1000 == 0:
        print(word, words2IDF[word])

##ck 7.3462626324699905
race 6.744087230115771
628 7.842699518783881
##medical 8.179171755405095
restricted 6.087307693726701
##enic 9.09546248727925
coronavirus 9.09546248727925
hex 8.535846699343827
oe 6.631609246689082


In [23]:
len(words2IDF)

8477

In [24]:
def calc_idf_score(sentence):
    """
    Calculate the mean idf score for given sentence.

    :param sentence: input sentence
    :type sentence: str
    :return: mean idf score of sentence token
    """
    tokenzied_sentence = [s.lower() for s in tokenizer_labse.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    return score / len(tokenzied_sentence)

### Saving the IDF values dictionary

In [25]:
import pickle
with open('IDFs.pkl', 'wb') as f:
    pickle.dump(words2IDF, f)

## Methods

### DR. TEIT*

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

**NOTE: In `predict_DR_TEIT` you may see a diffrent notation (`alpha`) but they are the same.**

#### TF-IDF Transformation Matrix Fitting

In [26]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        title_to_domain[doc_idx2] = doc_idx1
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
             s = (multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['spans'][id_sp1]['text_sec'].strip())
             s = s.replace('!', '.')
             s = s.replace('! ', '.')
             s = s.replace('?', '.')
             s = s.replace('? ', '.')
             s = s.replace('\n', '.')
             m = s.split('.')
             #print(m)
             S = ""
             func = lambda w: w[:1].lower() + w[1:] if w else ''
             for word in m:
                 S = S + func(word) + ' '
             doc_texts_train.append(S)
            
doc_texts_train[10]

'the number of credits needed to provide benefits for your survivors depends on your age when you die  No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit  But , the younger a person is , the fewer credits they must have for family members to receive survivors benefits  '

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(strip_accents=None,
                                 analyzer='char',
                                 ngram_range=(4, 10),
                                 norm='l2',
                                 use_idf=True,
                                 smooth_idf=True)
tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [28]:
import pickle
with open('tfidfVectorizer.pkl', 'wb') as f:
    pickle.dump(tfidfVectorizer, f)

with open('tfidf_wm.pkl', 'wb') as f:
    pickle.dump(tfidf_wm, f)

clarifying the history 

In [29]:
def combine_sentences(s1, s2):
    separation_token = "  "
    return s1 + separation_token + s2


def construct_followup_dataset(filepath):
    import json
    with open(filepath, 'r') as f:
        multidoc2dial_dial_train = json.load(f)
    
    historys = []
    questions = []
    combined = []
    labels = []
    prev_docs = []
    current_docs = []
    prev_answers = []

    for domain in multidoc2dial_dial_train['dial_data']:
        for dial in multidoc2dial_dial_train['dial_data'][domain]:
            prev_doc = ''
            prev_question = ''
            prev_answer = ''
            for turn in dial['turns']:
                if turn['role'] == "user":
                    current_question = turn['utterance']
                    historys.append(prev_question)
                    questions.append(current_question)
                    
                    combined.append(combine_sentences(prev_question, current_question))

                    current_doc = turn['references'][0]['doc_id']
                    labels.append(int(current_doc==prev_doc))

                    prev_docs.append(prev_doc)
                    current_docs.append(current_doc)
                    prev_answers.append(prev_answer)

                    prev_doc, prev_question = current_doc, current_question
                else:
                    prev_answer = turn['utterance']
                    
    return historys, questions, combined, labels, prev_docs, current_docs, prev_answers

In [30]:
import pandas as pd

train_history, train_questions, train_combined, train_labels, train_prev_docs, train_current_docs, train_prev_answers = construct_followup_dataset('/content/multidoc2dial/multidoc2dial_dial_train.json')
test_history, test_questions, test_combined, test_labels, test_prev_docs, test_current_docs, test_prev_answers = construct_followup_dataset('/content/multidoc2dial/multidoc2dial_dial_validation.json')

train_dict_dataset = {"history":train_history, "question": train_questions, "combined": train_combined, "followup": train_labels, "prev_doc": train_prev_docs, "current_doc": train_current_docs, "prev_answer": train_prev_answers}
test_dict_dataset = {"history":test_history, "question": test_questions, "combined": test_combined, "followup": test_labels, "prev_doc": test_prev_docs, "current_doc": test_current_docs, "prev_answer": test_prev_answers}

train_df = pd.DataFrame(train_dict_dataset)
test_df = pd.DataFrame(test_dict_dataset)

In [31]:
max([len(x["combined"].split()) for _, x in test_df.iterrows()])

73

In [32]:
test_df

,history,question,combined,followup,prev_doc,current_doc,prev_answer
0,,My insurance ended so what should i do,My insurance ended so what should i do,0,,Top 5 DMV Mistakes and How to Avoid Them#3_0,
1,My insurance ended so what should i do,Don't do that I'll get insurance,My insurance ended so what should i do Don't ...,1,Top 5 DMV Mistakes and How to Avoid Them#3_0,Top 5 DMV Mistakes and How to Avoid Them#3_0,You will need to get insurance or we will susp...
2,Don't do that I'll get insurance,"I have, that is why I am here to clear that up...","Don't do that I'll get insurance I have, that...",1,Top 5 DMV Mistakes and How to Avoid Them#3_0,Top 5 DMV Mistakes and How to Avoid Them#3_0,"Okay, have you received a letter from the DMV ..."
3,"I have, that is why I am here to clear that up...",Thank you so much. After looking through these...,"I have, that is why I am here to clear that up...",0,Top 5 DMV Mistakes and How to Avoid Them#3_0,Help finding enough proof of ID#3_0,"Okay, we can take care of that"
4,Thank you so much. After looking through these...,"Great. I think that I can found some bills, of...",Thank you so much. After looking through these...,1,Help finding enough proof of ID#3_0,Help finding enough proof of ID#3_0,"Sure, it is. You can contact your college and ..."
...,...,...,...,...,...,...,...
4491,"If I am totally and permanently disabled, can ...",In this case I would not like,"If I am totally and permanently disabled, can ...",1,Total and Permanent Disability Discharge | Fed...,Total and Permanent Disability Discharge | Fed...,want to qualify for a TPD download?
4492,In this case I would not like,If I am a veteran whose application for discha...,In this case I would not like If I am a veter...,1,Total and Permanent Disability Discharge | Fed...,Total and Permanent Disability Discharge | Fed...,"Unfortunately, no relevant information is found."
4493,If I am a veteran whose application for discha...,"In addition, I need to learn about PSLF. What ...",If I am a veteran whose application for discha...,0,Total and Permanent Disability Discharge | Fed...,Public Service Loan Forgiveness | Federal Stud...,in this case it is not subject to a post-disch...
4494,"In addition, I need to learn about PSLF. What ...",null,"In addition, I need to learn about PSLF. What ...",1,Public Service Loan Forgiveness | Federal Stud...,Public Service Loan Forgiveness | Federal Stud...,"Are you employed by a U.S. federal, state, loc..."


#### DR. TEIT

In [33]:
def predict_DR_TEIT(queries, k=1, alpha=10):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """

    idf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    tfidf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)
        # coef = 2**(-i) * calc_idf_score(query)
        coef = calc_idf_score(query)
        coef_sum += coef

        idf_score += coef * query_sim
        tfidf_score += coef * np.squeeze(np.asarray(tfidf_wm @ tfidfVectorizer.transform([query]).todense().T))

    scores = (idf_score + alpha * tfidf_score) / coef_sum
    best_k_idx = scores.argsort()[::-1][:k]
    scores = scores[best_k_idx]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    return (scores, predictions)

## Test
In the test dataset we just picked ones with **user** turn.

In [34]:
import json
with open('multidoc2dial/multidoc2dial_dial_validation.json', 'r') as f:
    multidoc2dial_dial_train = json.load(f)

In [35]:
doc_sentence_test = []
doc_label_test = []
for doc_idx1 in multidoc2dial_dial_train['dial_data']:
    for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
        for turns in dial['turns']:
            if turns['role'] == "user":
                doc_sentence_test.append(turns['utterance'])
                doc_label_test.append(turns['references'][0]['doc_id'])

In [36]:
TEST_SIZE = len(doc_sentence_test)
TEST_SIZE

4496

In [ ]:
TEST_SIZE = TEST_SIZE // 20   #   For making it faster

### DR.TEIT

In [ ]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = test_df.loc[i].current_doc
    query = []
    if(test_df.loc[i].followup == 1):
        query.append(test_df.loc[i].prev_answer)
        query.append(test_df.loc[i].question)
        query.append(test_df.loc[i].history)
    else:
        query.append(test_df.loc[i].question)
    accs, preds = predict_DR_TEIT( query,
                                   k=30000,
                                   alpha=10)
    pred = []
    for i in range(len(preds)):
       ind = titles.index(preds[i])
       #print(ind)
       pred.append(parent_titles[ind])
   # ranks.append(1 / (pred.index(act_doc) + 1))
    if act_doc == pred[0]:
        prec_at_1 += 1
    if act_doc in pred[:5]:
        prec_at_5 += 1
    if act_doc in pred[:10]:
        prec_at_10 += 1
    if act_doc in pred[:50]:
        prec_at_50 += 1
    if act_doc in pred[:100]:
        prec_at_100 += 1
    if act_doc in pred[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

## Results

At last we have resutls as follows:


| Method | @1 | @5 | @10 | @50 | @100 | MRR (mean, var) |
|:------:|:------:|:------:|:-------:|:-------:|:--------:|:---:|
| IDF - vanilla | 13% | 30% | 39% | 64% | 83% | (0.22, 0.11) |
| IDF - power-order | 15% | 31% | 41% | 65% | 83% | (0.23, 0.12) |
| IDF - power-order (softmax) | 10.7% | 23% | 31% | 57.6% | 78% | (0.18, 0.09) |
| IDF - self-attention | 13.9% | 29% | 38% | 62% | 82% | (0.22, 0.11) |
| **DR. TEIT** | **61.6%** | **86%** | **91%** | **96%** | **98%** | **(0.72, 0.13)** |

It shows that title informations were not enough for document retrieval.

# drafts

In [ ]:
tfidf_wm.shape

(488, 1047632)

In [ ]:
answers = tfidfVectorizer.transform(["Original Card for a Foreign Born U.S. Citizen Adult",
                                     "Hello world from far beyound!"]).todense()
query = tfidfVectorizer.transform(["Hello!"]).todense()

In [ ]:
print(answers.shape, query.shape)

(2, 1047632) (1, 1047632)


In [ ]:
import numpy as np
answers_sim = np.squeeze(np.asarray(tfidf_wm @ answers.T))
query_sim = np.squeeze(np.asarray(tfidf_wm @ query.T))

In [ ]:
print(answers_sim.shape, query_sim.shape)

(488, 2) (488,)


In [ ]:
list(map(lambda x: np.dot(x, query_sim) /
        (np.linalg.norm(query_sim) * np.linalg.norm(x)),
        answers_sim.T))

[0.7506911990367025, 0.934114716518692]